# Question 1

In [32]:
import tensorflow as tf
import pathlib
import PIL

## 讀取資料集

In [ ]:
PATH_TO_DATASET_FOLDER = pathlib.Path("HW4_MNIST_train/4")
BATCH_SIZE = 32
IMG_HEIGHT = 180
IMG_WEIDTH = 180

RATIO = 0.8

list_ds = tf.data.Dataset.list_files(str(PATH_TO_DATASET_FOLDER/"*.png"))

def process_file(file_path):
    bounding_box_info_file = tf.strings.join([
        tf.strings.substr(file_path, 0, tf.strings.length(file_path)-len(".png")),
        ".txt"
    ])
    infos = tf.strings.split(tf.io.read_file(bounding_box_info_file), '\t')
    infos = tf.map_fn(lambda x:tf.strings.to_number(x, tf.float32), infos, tf.float32)

    return tf.image.decode_png(tf.io.read_file(file_path)) , infos

list_ds = list_ds.map(process_file)

train_size = int(RATIO * len(list_ds))

train_ds = list_ds.take(train_size)
test_ds = list_ds.skip(train_size)

## 建構與訓練模型

## 讀取模型

## 呈現結果
Confusion Matrix

Top-1 & top-3 accuracy

Intersection over Union

Network Architecture

# Question 2
## 讀取資料集

## 建構模型

## 讀取模型

## 呈現結果